Aluno: Pedro Rodrigues Corrêa - 243236

## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [ ]:
!wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
!wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

--2024-04-17 18:14:50--  https://www.gutenberg.org/ebooks/67724.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-04-17 18:14:51--  http://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-04-17 18:14:52--  https://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 372878 (364K) [text/plain]
Saving to: ‘67724.txt.utf-8.17’

67724.txt.utf-8.17  100%[===================>] 

# Parâmetros

In [ ]:
context_size = 9
dropout = 0.1
embedding_dim = 64
hidden = 200
batch_size = 32
epochs = 20
alpha = 2
rank = 1
vocab_size = 20000 # Pode ser que não se chegue nesse valor

# Pacotes e Explorando Dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import time
import torch.nn.functional as F

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
text1 = open("67724.txt.utf-8","r").read()
text1_pt = text1.split("\n\n")[35:2675]

text2 = open("67725.txt.utf-8","r").read()
text2_pt = text2.split("\n\n")[32:2191]

paragraphs = text1_pt + text2_pt

cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

len(paragraphs), len(cleaned_paragraphs)

(4799, 4740)

In [ ]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

for _ in cleaned_paragraphs[100:110]:
  print(_)

len(cleaned_paragraphs)

--É verdade! acudio Loredano com uma ingenuidade simulada; isto me faz lembrar que só nos demorámos no Rio de Janeiro cinco dias, quando das outras vezes erão nunca menos de dez e quinze.
--Tive ordem para haver-me com toda a rapidez; e creio, continuou fitando no italiano um olhar severo, que não devo contas de minhas acções senão áquelles a quem dei o direito de pedi-las.
--_Per Bacco_, cavalheiro! Tomais as cousas ao revez. Ninguem vos pergunta por que motivo fazeis aquillo que vos praz: mas tambem achareis justo que cada um pense á sua maneira.
--Pensai o que quizerdes! disse Alvaro levantando os hombros e avançando o passo da sua cavalgadura.
A conversa interrompeo-se.
Os dous cavalleiros, um pouco adiantados ao resto do troço, caminhavão silenciosos uma par do outro.
Alvaro ás vezes enfiava o olhar pelo caminho como para medir a distancia que ainda tinhão de percorrer, e outras vezes parecia pensativo e preoccupado.
Nestas occasiões, o italiano lançava sobre elle um olhar á furto

4740

## Análise do dataset

In [ ]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+|[.,!?-]', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)
print(word_counts)

Counter({',': 7367, '.': 6140, '-': 6079, 'a': 4458, 'que': 4329, 'o': 4051, 'de': 3939, 'e': 3600, 'se': 2395, 'um': 1708, 'do': 1397, 'não': 1279, 'uma': 1247, 'da': 1132, 'os': 1111, 'com': 1013, 'sua': 925, 'para': 853, 'seu': 777, '!': 770, 'pery': 729, 'em': 721, 'as': 702, 'no': 638, '?': 628, 'por': 621, 'como': 594, 'ao': 591, 'lhe': 557, 'd': 491, 'á': 490, 'tinha': 478, 'era': 467, 'cecilia': 457, 'na': 452, 'é': 439, 'sobre': 415, 'mas': 410, 'elle': 405, 'dos': 372, 'indio': 340, 'me': 325, 'seus': 324, 'mais': 317, 'antonio': 303, 'quando': 287, 'alvaro': 278, 'disse': 258, 'das': 256, 'vos': 254, 'ella': 233, 'olhos': 227, 'te': 227, 'senhora': 227, 'menina': 215, 'pela': 213, 'tu': 204, 'nos': 200, 'depois': 198, 'isabel': 197, 'havia': 195, 'fidalgo': 194, 'casa': 192, 'estava': 187, 'ainda': 184, 'tempo': 180, 'mariz': 180, 'já': 180, 'momento': 174, 'loredano': 174, 'só': 173, 'aventureiros': 173, 'mesmo': 173, 'italiano': 173, 'todos': 170, 'pelo': 169, 'vida': 168,

## Criando um vocabulário

In [ ]:
UNK = '<UNK>'
most_frequent_words = [UNK] + [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words)}
vocab_size = len(vocab)

In [ ]:
len(vocab)

11843

In [ ]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+|[.,!?-]', sentence.lower())]

def decode_sentence(encoded_sentence, vocab):
    words = []
    # Iterate through each index in the encoded sentence
    for index in encoded_sentence:
        # Find the corresponding word in the vocabulary for the index
        # If the index is not found in the vocabulary, replace it with "<UNK>"
        word = next((word for word, code in vocab.items() if code == index), "<UNK>")
        words.append(word)
    return words

## Dataset

In [ ]:
def gera_input_target(text, context_size):
    contexts = []
    targets = []

    for i in range(len(text) - context_size):
        context = text[i: i + context_size]
        target = text[i + context_size]
        contexts.append(context)
        targets.append(target)

    return contexts, targets

exemplo = 'Hoje vai chover muito.'.split()

for n in range(1,4):
  inputs, targets = gera_input_target(exemplo, n)

  for input_target in zip(inputs, targets):
    print(f'{input_target[0]} -> {input_target[1]}')
  print('------------------------------')

['Hoje'] -> vai
['vai'] -> chover
['chover'] -> muito.
------------------------------
['Hoje', 'vai'] -> chover
['vai', 'chover'] -> muito.
------------------------------
['Hoje', 'vai', 'chover'] -> muito.
------------------------------


In [ ]:
class MyDataset(Dataset):
    def __init__(self, text, vocab, context_size=9, vocab_size = 3000):
        #self.text = cleaned_paragraphs
        self.vocab = vocab
        self.context_size = context_size
        self.data = [encode_sentence(sentence, self.vocab) for sentence in text]

        contexts_list = []
        targets_list = []

        for coded in self.data:
          if len(coded) < self.context_size:
            continue
          inputs, targets = gera_input_target(coded, context_size)
          contexts_list.extend(inputs)
          targets_list.extend(targets)

        self.contexts_tensor = torch.tensor(contexts_list)
        self.targets_tensor = torch.tensor(targets_list)


    def __len__(self):
        return len(self.targets_tensor)

    def __getitem__(self, idx):
        return self.contexts_tensor[idx], self.targets_tensor[idx]

In [ ]:
from sklearn.model_selection import train_test_split

train_text, test_text = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = MyDataset(train_text, vocab)
test_dataset = MyDataset(test_text, vocab)

In [ ]:
# Criando instâncias de DataLoader para treinamento e validação
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)
sample = next(iter(train_loader))
sample

[tensor([[ 2137,    11,  2769,     1,    68,    12,   153,     6,  3735],
         [    8,  7547,    16,     6,    88,     4,   202,    11,  1724],
         [  845,     8,  1851,     1,    64,   234,  2341,     1,  4236],
         [    6,  1912,   540,     1,    32,  5218,     6,  2847,     7],
         [    1,  1206,    28,    73,    66,     6,    19,   394,  5050],
         [ 1844,     7,   769,   510,     7,   502,     1,   976,    16],
         [ 4012,     2,     7, 11789, 11790,  5867,  5868,  4016,    98],
         [  477,     4,    89,    37,     4,   564,    11,  1181,     1],
         [   14,   711,     8,    12,  2174,     6,    66,    22,  6575],
         [    1,    27,    15,   387,     5,   980,    13,   687,     1],
         [    3,     9,  3085,     4,     9,  3074,   302,   193,     1],
         [    1,    13,   947,  4331,    16,     6,  4245,    14,  2594],
         [    3,     3,     8,   115,    20,  1182,     3,    53,     8],
         [    1,    48,    10,  6747, 

## Model

In [ ]:
class LanguageModel(torch.nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.LoRA_enabled = False # Default
        self.vocab_size = vocab_size

        # LoRA parameters
        self.lora_alpha = alpha
        self.lora_r = rank
        self.scaling = self.lora_alpha / self.lora_r

        # Embeddings layer
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        # First Linear Layer
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden, bias=True)
        # Activation and Dropout
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(dropout)
        # Second Linear Layer
        self.linear2 = nn.Linear(hidden, vocab_size+1, bias=True)

        # LoRA Matrixes
        # LoRA on embeddings layer
        self.embeddings_lora_B = nn.Parameter(torch.zeros(vocab_size+1, self.lora_r), requires_grad=False)
        self.embeddings_lora_A = nn.Parameter(torch.randn(self.lora_r, embedding_dim), requires_grad=False)
        # LoRA on the first linear layer
        self.linear1_lora_B = nn.Parameter(torch.zeros(context_size*embedding_dim, self.lora_r), requires_grad=False)
        self.linear1_lora_A = nn.Parameter(torch.randn(self.lora_r, hidden), requires_grad=False)
        # LoRA on the second linear layer
        self.linear2_lora_B = nn.Parameter(torch.zeros(hidden, self.lora_r), requires_grad=False)
        self.linear2_lora_A = nn.Parameter(torch.randn(self.lora_r, vocab_size+1), requires_grad=False)

    def forward(self, inputs):
        # Embeddings
        embeds = self.embeddings(inputs)
        if (self.LoRA_enabled):
            one_hot = F.one_hot(inputs, self.vocab_size+1).to(torch.float32)
            embeddings_LoRA = one_hot @ (self.embeddings_lora_B @ self.embeddings_lora_A)
            embeddings_LoRA = embeddings_LoRA * self.scaling
            embeds = embeds + embeddings_LoRA

        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)

        # First linear layer
        out = self.linear1(embeds)
        if (self.LoRA_enabled):
            linear1_lora_out = embeds @ (self.linear1_lora_B @ self.linear1_lora_A)
            linear1_lora_out = linear1_lora_out * self.scaling
            out = out + linear1_lora_out

        activation = self.tanh(out)
        activation = self.dropout(activation)

        # Second linear layer
        out = self.linear2(activation)
        if (self.LoRA_enabled):
            linear2_lora_out = activation @ (self.linear2_lora_B @ self.linear2_lora_A)
            linear2_lora_out = linear2_lora_out * self.scaling
            out = out + linear2_lora_out

        return out

    def enable_LoRA(self):
        self.LoRA_enabled = True
        # Freeze base model parameters
        print("Freezing Embeddings")
        self.embeddings.weight.requires_grad = False
        print("Freezing Layer 1")
        self.linear1.weight.requires_grad = False
        print("Freezing Layer 2")
        self.linear2.weight.requires_grad = False
        # Unfreeze LoRA parameters
        print("Unfreezing LoRA parameters")
        self.embeddings_lora_A.requires_grad = True
        self.embeddings_lora_B.requires_grad = True
        self.linear1_lora_A.requires_grad = True
        self.linear1_lora_B.requires_grad = True
        self.linear2_lora_A.requires_grad = True
        self.linear2_lora_B.requires_grad = True

    def disable_LoRA(self):
        self.LoRA_enabled = False
        print("Unfreezing Embeddings")
        self.embeddings.weight.requires_grad = True
        print("Unfreezing Layer 1")
        self.linear1.weight.requires_grad = True
        print("Unfreezing Layer 2")
        self.linear2.weight.requires_grad = True
        print("Freezing LoRA parameters")
        self.embeddings_lora_A.requires_grad = False
        self.embeddings_lora_B.requires_grad = False
        self.linear1_lora_A.requires_grad = False
        self.linear1_lora_B.requires_grad = False
        self.linear2_lora_A.requires_grad = False
        self.linear2_lora_B.requires_grad = False


    def apply_LoRA_weights(self):
        # Apply LoRA weights to the main model.

        lora_embeddings_weights = (self.embeddings_lora_B @ self.embeddings_lora_A) * self.scaling
        lora_linear1_weights = (self.linear1_lora_B @ self.linear1_lora_A).transpose(0, 1) * self.scaling
        lora_linear2_weights = (self.linear2_lora_B @ self.linear2_lora_A).transpose(0, 1) * self.scaling
        self.embeddings.weight.data += lora_embeddings_weights
        self.linear1.weight.data += lora_linear1_weights
        self.linear2.weight.data += lora_linear2_weights

In [ ]:
model = LanguageModel()


In [ ]:
print(model)

LanguageModel(
  (embeddings): Embedding(11844, 64)
  (linear1): Linear(in_features=576, out_features=200, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=200, out_features=11844, bias=True)
)


In [ ]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

output = model(input)
pred = output.argmax(dim=1)

print(pred)
print(target)

torch.Size([32, 9])
torch.Size([32])
tensor([ 4186,  3663,   203,  8856,  3537,  8653,  6430, 11618,   287,  4505,
         3770,   947,   418,  6641,  7205,  5480,   938,  7275,  9403,  8206,
        11673,  9251, 11705,   248,  7233,  9229,  6173,  1583,  4143,  7972,
         3774,  8361])
tensor([   8,   25,   57,   41,    2,    1,   11,  159,  401, 3172, 1010,  188,
         364,   13,  107,    5,    8,    8, 5633,   87,   76,    8,   21,   81,
          14,   54,  100,  325,  141,   85,  419,   44])


In [ ]:
print(model(input).shape)

torch.Size([32, 11844])


In [ ]:
output.argmax(dim=1)

tensor([ 4186,  3663,   203,  8856,  3537,  8653,  6430, 11618,   287,  4505,
         3770,   947,   418,  6641,  7205,  5480,   938,  7275,  9403,  8206,
        11673,  9251, 11705,   248,  7233,  9229,  6173,  1583,  4143,  7972,
         3774,  8361])

In [ ]:
target

tensor([   8,   25,   57,   41,    2,    1,   11,  159,  401, 3172, 1010,  188,
         364,   13,  107,    5,    8,    8, 5633,   87,   76,    8,   21,   81,
          14,   54,  100,  325,  141,   85,  419,   44])

In [ ]:
# Conta o número de parâmetros
def get_num_params(model):
    n_params = sum(p.numel() for p in model.parameters())
    n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'O modelo tem um total de {n_params}, dos quais {n_trainable_params} são treináveis.')

# Funções utilizadas no treinamento do modelo

In [ ]:
def init_eval(model):
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()

    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

In [ ]:
def train(model):
  for epoch in range(epochs):

    model.train()

    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    model.eval()

    with torch.no_grad():
        eval_loss = 0

        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            eval_loss += criterion(outputs, targets)

        eval_loss = eval_loss / len(test_loader)


    print(f'Epoch [{epoch+1}/{epochs}], \
        Train Loss: {loss.item():.4f}, \
        Train PPL: {torch.exp(loss):.4f}, \
        Eval Loss: {eval_loss.item():.4f}, \
        Eval PPL: {torch.exp(eval_loss):.4f}')

In [ ]:
def eval(model):
  model.eval()

  with torch.no_grad():
      eval_loss = 0

      for inputs, targets in test_loader:
          inputs = inputs.to(device)
          targets = targets.to(device)

          outputs = model(inputs)
          eval_loss += criterion(outputs, targets)

      eval_loss = eval_loss / len(test_loader)


  print(f'Eval Loss: {eval_loss.item():.4f}, \
      Eval PPL: {torch.exp(eval_loss):.4f}')

# Treinamento sem LoRA

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
model.to(device)

model.to(device)
print(model)
get_num_params(model)

LanguageModel(
  (embeddings): Embedding(11844, 64)
  (linear1): Linear(in_features=576, out_features=200, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=200, out_features=11844, bias=True)
)
O modelo tem um total de 3278788, dos quais 3254060 são treináveis.


In [ ]:
print("Base Model - No LoRA")
print()
print("Initial Evaluation")
print()
init_eval(model)
print()
print("Training the Model")
print()
train(model)
print()
print("Evaluation on the Validation Dataset")
eval(model)

Base Model - No LoRA

Initial Evaluation

Initial Loss: 9.4292
Initial Perplexity: 12446.0693

Training the Model

Epoch [1/20],         Train Loss: 9.2997,         Train PPL: 10934.5693,         Eval Loss: 9.2738,         Eval PPL: 10655.6914
Epoch [2/20],         Train Loss: 9.4197,         Train PPL: 12328.8320,         Eval Loss: 8.9840,         Eval PPL: 7974.3916
Epoch [3/20],         Train Loss: 9.2746,         Train PPL: 10663.9463,         Eval Loss: 8.3792,         Eval PPL: 4355.7188
Epoch [4/20],         Train Loss: 5.2474,         Train PPL: 190.0622,         Eval Loss: 7.8465,         Eval PPL: 2556.6917
Epoch [5/20],         Train Loss: 9.2798,         Train PPL: 10719.3232,         Eval Loss: 7.6045,         Eval PPL: 2007.2440
Epoch [6/20],         Train Loss: 9.6970,         Train PPL: 16269.4805,         Eval Loss: 7.4345,         Eval PPL: 1693.3337
Epoch [7/20],         Train Loss: 6.2628,         Train PPL: 524.6857,         Eval Loss: 7.3008,         Eval PPL: 14

# Treinamento com LoRA (fine-tuning)

In [ ]:
model.enable_LoRA()
print(model)
get_num_params(model)

Freezing Embeddings
Freezing Layer 1
Freezing Layer 2
Unfreezing LoRA parameters
LanguageModel(
  (embeddings): Embedding(11844, 64)
  (linear1): Linear(in_features=576, out_features=200, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=200, out_features=11844, bias=True)
)
O modelo tem um total de 3278788, dos quais 36772 são treináveis.


In [ ]:
# Treinamento usando o LoRA após o pré-treinado sem ele
epochs = 10

print("Base Model - With LoRA")
print()
print("Training the Model")
print()
train(model)
print()
print("Evaluation on the Validation Dataset")
eval(model)

Base Model - With LoRA

Training the Model

Epoch [1/10],         Train Loss: 7.1461,         Train PPL: 1269.1721,         Eval Loss: 6.6637,         Eval PPL: 783.4623
Epoch [2/10],         Train Loss: 5.5544,         Train PPL: 258.3614,         Eval Loss: 6.6459,         Eval PPL: 769.6369
Epoch [3/10],         Train Loss: 10.1409,         Train PPL: 25358.3262,         Eval Loss: 6.6398,         Eval PPL: 764.9428
Epoch [4/10],         Train Loss: 9.9424,         Train PPL: 20794.5605,         Eval Loss: 6.7065,         Eval PPL: 817.6917
Epoch [5/10],         Train Loss: 11.2978,         Train PPL: 80647.8750,         Eval Loss: 6.7998,         Eval PPL: 897.6899
Epoch [6/10],         Train Loss: 6.8673,         Train PPL: 960.3467,         Eval Loss: 6.6436,         Eval PPL: 767.8748
Epoch [7/10],         Train Loss: 3.4289,         Train PPL: 30.8427,         Eval Loss: 6.6438,         Eval PPL: 768.0172
Epoch [8/10],         Train Loss: 9.7352,         Train PPL: 16902.1191, 

# Consolidação

In [ ]:
model.disable_LoRA()
model.apply_LoRA_weights()
print(model)
get_num_params(model)

Unfreezing Embeddings
Unfreezing Layer 1
Unfreezing Layer 2
Freezing LoRA parameters
LanguageModel(
  (embeddings): Embedding(11844, 64)
  (linear1): Linear(in_features=576, out_features=200, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=200, out_features=11844, bias=True)
)
O modelo tem um total de 3278788, dos quais 3254060 são treináveis.


In [ ]:
print("Modelo Final com pesos W + LoRA")
print()
print()
print("Evaluation on the Validation Dataset")
eval(model)

Modelo Final com pesos W + LoRA


Evaluation on the Validation Dataset
Eval Loss: 6.6737,       Eval PPL: 791.2866


## Referências

Fábio Grassiotto